# Lesson 6: Constraint Polynomials

In this lesson, we'll examine how the rule checks from Lesson 2 are transformed into polynomial constraints. This is a key step in creating a STARK, as it allows us to verify computation correctness algebraically.

In [ ]:
import numpy as np
from sympy import symbols, expand, Poly
import matplotlib.pyplot as plt

def create_constraint_polynomial(reg1, reg2, reg3):
    """Creates a constraint polynomial for the Fibonacci rule.
    
    Args:
        reg1, reg2, reg3: Register values
        
    Returns:
        sympy.Poly: Constraint polynomial
    """
    x = symbols('x')
    # Rule: reg3 = (reg1 + reg2) % 97
    constraint = reg1 + reg2 - reg3
    return Poly(constraint, x)

## Creating and Visualizing Polynomial Constraints

In [ ]:
# Import functions from previous lessons
from lesson_1_execution_trace import create_fibonacci_trace
from lesson_3_padding_trace import pad_trace_to_power_of_two
from lesson_4_trace_polynomials import create_trace_polynomial

# Create trace
trace = create_fibonacci_trace(3, 5)
padded_trace = pad_trace_to_power_of_two(trace)

# Create domain for interpolation
n_points = len(padded_trace)
domain = np.linspace(-1, 1, n_points)

# Create polynomials for registers
polynomials = {}
for column in ['Register 1', 'Register 2', 'Register 3']:
    values = padded_trace[column].values
    poly = create_trace_polynomial(values, domain)
    polynomials[column] = poly

# Create and evaluate constraint polynomial
constraint_values = []
for i in range(len(domain)):
    values = [polynomials[col](domain[i]) for col in ['Register 1', 'Register 2', 'Register 3']]
    constraint = create_constraint_polynomial(*values)
    constraint_values.append(float(constraint.eval(domain[i])))

# Visualization
plt.figure(figsize=(15, 6))

# Plot constraint values
plt.plot(domain, constraint_values, 'r-', label='Constraint Polynomial')
plt.axhline(y=0, color='k', linestyle='--', alpha=0.3)

plt.title('Constraint Polynomial Values')
plt.xlabel('Domain')
plt.ylabel('Constraint Value')
plt.grid(True)
plt.legend()
plt.show()

## Verifying Constraint Satisfaction

In [ ]:
def verify_constraints(polynomials, domain):
    """Verifies constraint satisfaction at all domain points.
    
    Args:
        polynomials: Dictionary of register polynomials
        domain: Domain points
    """
    max_violation = 0
    violation_points = []
    
    for i, x in enumerate(domain):
        values = [polynomials[col](x) for col in ['Register 1', 'Register 2', 'Register 3']]
        constraint = create_constraint_polynomial(*values)
        violation = abs(float(constraint.eval(x)))
        
        if violation > 1e-10:  # Account for numerical precision
            max_violation = max(max_violation, violation)
            violation_points.append((x, violation))
    
    print(f"Maximum constraint violation: {max_violation:.2e}")
    if violation_points:
        print("\nPoints with constraint violations:")
        for x, v in violation_points:
            print(f"x = {x:.2f}, violation = {v:.2e}")
    else:
        print("\nAll constraints satisfied!")

# Check constraints
verify_constraints(polynomials, domain)

## Important Aspects of Polynomial Constraints

1. **Algebraic Form**:
   - Each rule check is transformed into a polynomial
   - Polynomial must equal zero at all domain points
   - Constraint polynomial degree affects proof efficiency

2. **Advantages of the Approach**:
   - Enables algebraic methods for verification
   - Simplifies zero-knowledge proof creation
   - Provides compact representation

3. **Practical Considerations**:
   - Must account for numerical precision
   - Important to choose appropriate interpolation domain
   - Polynomial degree affects performance

In the next lesson, we'll examine how these polynomial constraints are combined to create a unified proof.